In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
type(tables[0])

bs4.element.Tag

In [7]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [8]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [9]:
df

Opening  Opening.1                  Title  \
0     JANUARY          3             The Grudge   
1     JANUARY         10             Underwater   
2     JANUARY         10            Like a Boss   
3     JANUARY         10          Three Christs   
4     JANUARY         10      Inherit the Viper   
..        ...        ...                    ...   
267  DECEMBER         25       We Can Be Heroes   
268  DECEMBER         25      News of the World   
269  DECEMBER         25  One Night in Miami...   
270  DECEMBER         25  Promising Young Woman   
271  DECEMBER         30      Pieces of a Woman   

                                    Production company  \
0    Screen Gems / Stage 6 Films / Ghost House Pict...   
1    20th Century Fox / TSG Entertainment / Chernin...   
2                                   Paramount Pictures   
3                                            IFC Films   
4       Barry Films / Tycor International Film Company   
..                                                 ...   
267                     Netflix / Troublemaker Studios   
268        Universal Pictures / Perfect World Pictures   
269                                     Amazon Studios   
270          Focus Features / FilmNation Entertainment   
271                             Netflix / Bron Studios   

                                         Cast and crew  \
0    Nicolas Pesce (director/screenplay); Andrea Ri...   
1    William Eubank (director); Brian Duffield, Ada...   
2    Miguel Arteta (director); Sam Pitman, Adam Col...   
3    Jon Avnet (director/screenplay); Eric Nazarian...   
4    Anthony Jerjen (director); Andrew Crabtree (sc...   
..                                                 ...   
267  Robert Rodriguez (director/screenplay); Priyan...   
268  Paul Greengrass (director/screenplay); Luke Da...   
269  Regina King (director); Kemp Powers (screenpla...   
270  Emerald Fennell (director/screenplay); Carey M...   
271  Kornél Mundruczó (director); Kata Wéber (scree...   

    .mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.  \
0                                                  [2]                            
1                                                  [3]                            
2                                                  [4]                            
3                                                  NaN                            
4                                                  [5]                            
..                                                 ...                            
267                                                NaN                            
268                                                NaN                            
269                                                NaN                            
270                                                NaN                            
271                                                NaN                            

      Ref.  
0      NaN  
1      NaN  
2      NaN  
3      NaN  
4      NaN  
..     ...  
267  [240]  
268  [241]  
269  [242]  
270  [243]  
271  [244]  

[272 rows x 7 columns]

In [10]:
df_2020 = df[['Title','Cast and crew']]

In [11]:
df_2020

Title                                      Cast and crew
0               The Grudge  Nicolas Pesce (director/screenplay); Andrea Ri...
1               Underwater  William Eubank (director); Brian Duffield, Ada...
2              Like a Boss  Miguel Arteta (director); Sam Pitman, Adam Col...
3            Three Christs  Jon Avnet (director/screenplay); Eric Nazarian...
4        Inherit the Viper  Anthony Jerjen (director); Andrew Crabtree (sc...
..                     ...                                                ...
267       We Can Be Heroes  Robert Rodriguez (director/screenplay); Priyan...
268      News of the World  Paul Greengrass (director/screenplay); Luke Da...
269  One Night in Miami...  Regina King (director); Kemp Powers (screenpla...
270  Promising Young Woman  Emerald Fennell (director/screenplay); Carey M...
271      Pieces of a Woman  Kornél Mundruczó (director); Kata Wéber (scree...

[272 rows x 2 columns]

In [12]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = 'f530dee3755d9b106f916a9d8b2b9c26'

In [13]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [14]:
df_2020['genres'] = df_2020['Title'].map(lambda x: get_genre(str(x)))

<ipython-input-14-fdc91464740e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['genres'] = df_2020['Title'].map(lambda x: get_genre(str(x)))


In [15]:
df_2020

Title                                      Cast and crew  \
0               The Grudge  Nicolas Pesce (director/screenplay); Andrea Ri...   
1               Underwater  William Eubank (director); Brian Duffield, Ada...   
2              Like a Boss  Miguel Arteta (director); Sam Pitman, Adam Col...   
3            Three Christs  Jon Avnet (director/screenplay); Eric Nazarian...   
4        Inherit the Viper  Anthony Jerjen (director); Andrew Crabtree (sc...   
..                     ...                                                ...   
267       We Can Be Heroes  Robert Rodriguez (director/screenplay); Priyan...   
268      News of the World  Paul Greengrass (director/screenplay); Luke Da...   
269  One Night in Miami...  Regina King (director); Kemp Powers (screenpla...   
270  Promising Young Woman  Emerald Fennell (director/screenplay); Carey M...   
271      Pieces of a Woman  Kornél Mundruczó (director); Kata Wéber (scree...   

                                     genres  
0                   Horror Mystery Thriller  
1    Action Horror Science Fiction Thriller  
2                                    Comedy  
3                                     Drama  
4                      Crime Thriller Drama  
..                                      ...  
267            Action Fantasy Family Comedy  
268                       Adventure Western  
269                                   Drama  
270                    Thriller Crime Drama  
271                                   Drama  

[272 rows x 3 columns]

In [16]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [17]:
df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))

<ipython-input-17-bc52fe9b0c35>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))


In [18]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [19]:
df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))

<ipython-input-19-42c5f763f942>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [20]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [21]:
df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))

<ipython-input-21-ca9c918a7840>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))


In [22]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [23]:
df_2020['actor_3_name'] = df_2020['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [24]:
df_2020

Title                                      Cast and crew  \
0               The Grudge  Nicolas Pesce (director/screenplay); Andrea Ri...   
1               Underwater  William Eubank (director); Brian Duffield, Ada...   
2              Like a Boss  Miguel Arteta (director); Sam Pitman, Adam Col...   
3            Three Christs  Jon Avnet (director/screenplay); Eric Nazarian...   
4        Inherit the Viper  Anthony Jerjen (director); Andrew Crabtree (sc...   
..                     ...                                                ...   
267       We Can Be Heroes  Robert Rodriguez (director/screenplay); Priyan...   
268      News of the World  Paul Greengrass (director/screenplay); Luke Da...   
269  One Night in Miami...  Regina King (director); Kemp Powers (screenpla...   
270  Promising Young Woman  Emerald Fennell (director/screenplay); Carey M...   
271      Pieces of a Woman  Kornél Mundruczó (director); Kata Wéber (scree...   

                                     genres     director_name  \
0                   Horror Mystery Thriller     Nicolas Pesce   
1    Action Horror Science Fiction Thriller    William Eubank   
2                                    Comedy     Miguel Arteta   
3                                     Drama         Jon Avnet   
4                      Crime Thriller Drama    Anthony Jerjen   
..                                      ...               ...   
267            Action Fantasy Family Comedy  Robert Rodriguez   
268                       Adventure Western   Paul Greengrass   
269                                   Drama       Regina King   
270                    Thriller Crime Drama   Emerald Fennell   
271                                   Drama  Kornél Mundruczó   

              actor_1_name       actor_2_name     actor_3_name  
0       Andrea Riseborough      Demián Bichir         John Cho  
1          Kristen Stewart     Vincent Cassel  Jessica Henwick  
2          Tiffany Haddish         Rose Byrne      Salma Hayek  
3             Richard Gere     Peter Dinklage   Walton Goggins  
4            Josh Hartnett  Margarita Levieva   Chandler Riggs  
..                     ...                ...              ...  
267  Priyanka Chopra Jonas       Pedro Pascal    YaYa Gosselin  
268              Tom Hanks      Helena Zengel              NaN  
269      Kingsley Ben-Adir          Eli Goree      Aldis Hodge  
270         Carey Mulligan         Bo Burnham      Alison Brie  
271          Vanessa Kirby       Shia LaBeouf     Molly Parker  

[272 rows x 7 columns]

In [25]:
df_2020 = df_2020.rename(columns={'Title':'movie_title'})

In [26]:
new_df20 = df_2020.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [27]:
new_df20

director_name           actor_1_name       actor_2_name  \
0       Nicolas Pesce     Andrea Riseborough      Demián Bichir   
1      William Eubank        Kristen Stewart     Vincent Cassel   
2       Miguel Arteta        Tiffany Haddish         Rose Byrne   
3           Jon Avnet           Richard Gere     Peter Dinklage   
4      Anthony Jerjen          Josh Hartnett  Margarita Levieva   
..                ...                    ...                ...   
267  Robert Rodriguez  Priyanka Chopra Jonas       Pedro Pascal   
268   Paul Greengrass              Tom Hanks      Helena Zengel   
269       Regina King      Kingsley Ben-Adir          Eli Goree   
270   Emerald Fennell         Carey Mulligan         Bo Burnham   
271  Kornél Mundruczó          Vanessa Kirby       Shia LaBeouf   

        actor_3_name                                  genres  \
0           John Cho                 Horror Mystery Thriller   
1    Jessica Henwick  Action Horror Science Fiction Thriller   
2        Salma Hayek                                  Comedy   
3     Walton Goggins                                   Drama   
4     Chandler Riggs                    Crime Thriller Drama   
..               ...                                     ...   
267    YaYa Gosselin            Action Fantasy Family Comedy   
268              NaN                       Adventure Western   
269      Aldis Hodge                                   Drama   
270      Alison Brie                    Thriller Crime Drama   
271     Molly Parker                                   Drama   

               movie_title  
0               The Grudge  
1               Underwater  
2              Like a Boss  
3            Three Christs  
4        Inherit the Viper  
..                     ...  
267       We Can Be Heroes  
268      News of the World  
269  One Night in Miami...  
270  Promising Young Woman  
271      Pieces of a Woman  

[272 rows x 6 columns]

In [28]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']

In [29]:
new_df20.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      4
actor_3_name     27
genres            1
movie_title       0
comb             28
dtype: int64

In [30]:
new_df20 = new_df20.dropna(how='any')

In [31]:
new_df20.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [32]:
new_df20['movie_title'] = new_df20['movie_title'].str.lower()

<ipython-input-32-83e83e91073b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df20['movie_title'] = new_df20['movie_title'].str.lower()


In [33]:
new_df20

director_name           actor_1_name       actor_2_name  \
0       Nicolas Pesce     Andrea Riseborough      Demián Bichir   
1      William Eubank        Kristen Stewart     Vincent Cassel   
2       Miguel Arteta        Tiffany Haddish         Rose Byrne   
3           Jon Avnet           Richard Gere     Peter Dinklage   
4      Anthony Jerjen          Josh Hartnett  Margarita Levieva   
..                ...                    ...                ...   
266       Pete Docter             Jamie Foxx           Tina Fey   
267  Robert Rodriguez  Priyanka Chopra Jonas       Pedro Pascal   
269       Regina King      Kingsley Ben-Adir          Eli Goree   
270   Emerald Fennell         Carey Mulligan         Bo Burnham   
271  Kornél Mundruczó          Vanessa Kirby       Shia LaBeouf   

        actor_3_name                                  genres  \
0           John Cho                 Horror Mystery Thriller   
1    Jessica Henwick  Action Horror Science Fiction Thriller   
2        Salma Hayek                                  Comedy   
3     Walton Goggins                                   Drama   
4     Chandler Riggs                    Crime Thriller Drama   
..               ...                                     ...   
266    Graham Norton   Family Animation Music Comedy Fantasy   
267    YaYa Gosselin            Action Fantasy Family Comedy   
269      Aldis Hodge                                   Drama   
270      Alison Brie                    Thriller Crime Drama   
271     Molly Parker                                   Drama   

               movie_title                                               comb  
0               the grudge  Andrea Riseborough Demián Bichir John Cho Nico...  
1               underwater  Kristen Stewart Vincent Cassel Jessica Henwick...  
2              like a boss  Tiffany Haddish Rose Byrne Salma Hayek Miguel ...  
3            three christs  Richard Gere Peter Dinklage Walton Goggins Jon...  
4        inherit the viper  Josh Hartnett Margarita Levieva Chandler Riggs...  
..                     ...                                                ...  
266                   soul  Jamie Foxx Tina Fey Graham Norton Pete Docter ...  
267       we can be heroes  Priyanka Chopra Jonas Pedro Pascal YaYa Gossel...  
269  one night in miami...  Kingsley Ben-Adir Eli Goree Aldis Hodge Regina...  
270  promising young woman  Carey Mulligan Bo Burnham Alison Brie Emerald ...  
271      pieces of a woman  Vanessa Kirby Shia LaBeouf Molly Parker Kornél...  

[244 rows x 7 columns]

In [34]:
old_df = pd.read_csv('final_data.csv')

In [35]:
old_df

director_name       actor_1_name          actor_2_name  \
0              James Cameron        CCH Pounder      Joel David Moore   
1             Gore Verbinski        Johnny Depp         Orlando Bloom   
2                 Sam Mendes    Christoph Waltz          Rory Kinnear   
3          Christopher Nolan          Tom Hardy        Christian Bale   
4                Doug Walker        Doug Walker            Rob Walker   
...                      ...                ...                   ...   
5868  Nick Bruno, Troy Quane         Will Smith           Tom Holland   
5869            Greta Gerwig      Saoirse Ronan           Emma Watson   
5870              Sam Mendes      George MacKay  Dean-Charles Chapman   
5871   Destin Daniel Cretton  Michael B. Jordan            Jamie Foxx   
5872         Chinonye Chukwu      Alfre Woodard        Wendell Pierce   

              actor_3_name                                    genres  \
0                Wes Studi           Action Adventure Fantasy Sci-Fi   
1           Jack Davenport                  Action Adventure Fantasy   
2         Stephanie Sigman                 Action Adventure Thriller   
3     Joseph Gordon-Levitt                           Action Thriller   
4                  unknown                               Documentary   
...                    ...                                       ...   
5868         Rashida Jones  Animation Action Adventure Comedy Family   
5869         Florence Pugh                             Drama Romance   
5870           Mark Strong                 War Drama Action Thriller   
5871           Brie Larson                       Drama Crime History   
5872           Aldis Hodge                                     Drama   

                                            movie_title  \
0                                                avatar   
1              pirates of the caribbean: at world's end   
2                                               spectre   
3                                 the dark knight rises   
4     star wars: episode vii - the force awakens    ...   
...                                                 ...   
5868                                  spies in disguise   
5869                                       little women   
5870                                               1917   
5871                                         just mercy   
5872                                           clemency   

                                                   comb  
0     CCH Pounder Joel David Moore Wes Studi James C...  
1     Johnny Depp Orlando Bloom Jack Davenport Gore ...  
2     Christoph Waltz Rory Kinnear Stephanie Sigman ...  
3     Tom Hardy Christian Bale Joseph Gordon-Levitt ...  
4     Doug Walker Rob Walker unknown Doug Walker Doc...  
...                                                 ...  
5868  Will Smith Tom Holland Rashida Jones Nick Brun...  
5869  Saoirse Ronan Emma Watson Florence Pugh Greta ...  
5870  George MacKay Dean-Charles Chapman Mark Strong...  
5871  Michael B. Jordan Jamie Foxx Brie Larson Desti...  
5872  Alfre Woodard Wendell Pierce Aldis Hodge Chino...  

[5873 rows x 7 columns]

In [36]:
final_df = old_df.append(new_df20,ignore_index=True)

In [37]:
final_df

director_name           actor_1_name      actor_2_name  \
0         James Cameron            CCH Pounder  Joel David Moore   
1        Gore Verbinski            Johnny Depp     Orlando Bloom   
2            Sam Mendes        Christoph Waltz      Rory Kinnear   
3     Christopher Nolan              Tom Hardy    Christian Bale   
4           Doug Walker            Doug Walker        Rob Walker   
...                 ...                    ...               ...   
6112        Pete Docter             Jamie Foxx          Tina Fey   
6113   Robert Rodriguez  Priyanka Chopra Jonas      Pedro Pascal   
6114        Regina King      Kingsley Ben-Adir         Eli Goree   
6115    Emerald Fennell         Carey Mulligan        Bo Burnham   
6116   Kornél Mundruczó          Vanessa Kirby      Shia LaBeouf   

              actor_3_name                                 genres  \
0                Wes Studi        Action Adventure Fantasy Sci-Fi   
1           Jack Davenport               Action Adventure Fantasy   
2         Stephanie Sigman              Action Adventure Thriller   
3     Joseph Gordon-Levitt                        Action Thriller   
4                  unknown                            Documentary   
...                    ...                                    ...   
6112         Graham Norton  Family Animation Music Comedy Fantasy   
6113         YaYa Gosselin           Action Fantasy Family Comedy   
6114           Aldis Hodge                                  Drama   
6115           Alison Brie                   Thriller Crime Drama   
6116          Molly Parker                                  Drama   

                                            movie_title  \
0                                                avatar   
1              pirates of the caribbean: at world's end   
2                                               spectre   
3                                 the dark knight rises   
4     star wars: episode vii - the force awakens    ...   
...                                                 ...   
6112                                               soul   
6113                                   we can be heroes   
6114                              one night in miami...   
6115                              promising young woman   
6116                                  pieces of a woman   

                                                   comb  
0     CCH Pounder Joel David Moore Wes Studi James C...  
1     Johnny Depp Orlando Bloom Jack Davenport Gore ...  
2     Christoph Waltz Rory Kinnear Stephanie Sigman ...  
3     Tom Hardy Christian Bale Joseph Gordon-Levitt ...  
4     Doug Walker Rob Walker unknown Doug Walker Doc...  
...                                                 ...  
6112  Jamie Foxx Tina Fey Graham Norton Pete Docter ...  
6113  Priyanka Chopra Jonas Pedro Pascal YaYa Gossel...  
6114  Kingsley Ben-Adir Eli Goree Aldis Hodge Regina...  
6115  Carey Mulligan Bo Burnham Alison Brie Emerald ...  
6116  Vanessa Kirby Shia LaBeouf Molly Parker Kornél...  

[6117 rows x 7 columns]

In [38]:
final_df.to_csv('main_data.csv',index=False)